<a href="https://colab.research.google.com/github/narendraakumar/SDNAWORK/blob/main/duplicate_finder_(2)_(2)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install torch torchvision opencv-python opencv-python-headless matplotlib yolov5
!pip install grad-cam
!pip install rembg
!pip install onnxruntime
!git clone https://github.com/narendraakumar/Mask2Former.git
%cd Mask2Former
!pip install -U opencv-python
!pip install git+https://github.com/cocodataset/panopticapi.git
!pip install -r requirements.txt
%cd mask2former/modeling/pixel_decoder/ops
!python setup.py build install
%cd ../../../../

In [2]:
%%capture
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/narendraakumar/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

In [3]:
!wget https://filebin.net/archive/8ffetpsf8d0zp8c9/zip

--2024-11-28 05:42:11--  https://filebin.net/archive/8ffetpsf8d0zp8c9/zip
Resolving filebin.net (filebin.net)... 88.99.137.18, 2a01:4f8:10a:2156::2
Connecting to filebin.net (filebin.net)|88.99.137.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘zip’

zip                     [           <=>      ]   2.97G  21.0MB/s    in 2m 38s  

2024-11-28 05:44:51 (19.2 MB/s) - ‘zip’ saved [3188268126]



In [4]:
# You may need to restart your runtime prior to this, to let your installation take effect
%cd /content/detectron2
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
setup_logger(name="mask2former")

# import some common libraries
import numpy as np
import cv2
import torch
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2.projects.deeplab import add_deeplab_config
coco_metadata = MetadataCatalog.get("coco_2017_val_panoptic")

# import Mask2Former project
from mask2former import add_maskformer2_config

[Errno 2] No such file or directory: '/content/detectron2'
/content/Mask2Former


/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/content/Mask2Former/mask2former/modeling/pixel_decoder/msdeformattn.py:314: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


In [5]:
cfg = get_cfg()
add_deeplab_config(cfg)
add_maskformer2_config(cfg)
cfg.merge_from_file("configs/coco/panoptic-segmentation/swin/maskformer2_swin_large_IN21k_384_bs16_100ep.yaml")
cfg.MODEL.WEIGHTS = 'https://dl.fbaipublicfiles.com/maskformer/mask2former/coco/panoptic/maskformer2_swin_large_IN21k_384_bs16_100ep/model_final_f07440.pkl'
# cfg.MODEL.MASK_FORMER.TEST.SEMANTIC_ON = True
# cfg.MODEL.MASK_FORMER.TEST.INSTANCE_ON = True
cfg.MODEL.MASK_FORMER.TEST.PANOPTIC_ON = True
predictor = DefaultPredictor(cfg)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[11/28 05:45:10 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/maskformer/mask2former/coco/panoptic/maskformer2_swin_large_IN21k_384_bs16_100ep/model_final_f07440.pkl ...


model_final_f07440.pkl: 866MB [00:05, 162MB/s]                           


WARNING [11/28 05:45:17 mask2former.modeling.transformer_decoder.mask2former_transformer_decoder]: Weight format of MultiScaleMaskedTransformerDecoder have changed! Please upgrade your models. Applying automatic conversion now ...


In [ ]:
import os,shutil
from PIL import Image

def resize_image(im_path, output_base_dir="/content/tmp/"):
    """
    Resize an image if its width or height exceeds 1000 pixels and save to a new directory
    while keeping the folder structure the same.

    Args:
        im_path (str): Path to the original image.
        output_base_dir (str): Base directory for resized images.

    Returns:
        str: Path to the resized image or original image.
    """
    # Open the original image
    # if os.path.exists():
    # shutil.rmtree(output_base_dir)
    image = Image.open(im_path)

    # Check if resizing is needed
    if image.width > 1000 or image.height > 1000:
        # Calculate new dimensions
        new_width = int(image.width * 0.5)
        new_height = int(image.height * 0.5)

        # Resize using high-quality interpolation
        resized_image = image.resize((new_width, new_height), Image.LANCZOS)

        # Ensure the output directory structure is preserved
        parent_dir = os.path.join(im_path, os.pardir+"/"+os.pardir)
        relative_path = os.path.relpath(im_path, start=parent_dir)

        new_path = os.path.join(output_base_dir, relative_path)

        # Add file extension if it's missing
        if not os.path.splitext(new_path)[1]:
            new_path += os.path.splitext(im_path)[1]

        os.makedirs(os.path.dirname(new_path), exist_ok=True)

        # Save the resized image
        print(new_path,relative_path)
        resized_image.save(new_path)
        print(f"Image resized and saved: {new_path}")
        return new_path
    else:
        print("Image dimensions are below 1000px; no resizing performed.")
        return im_path

In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow


class Img:
    def __init__(self, img_path=None,img=None):

        self.img_path = img_path
        if img is not None:
            self.image = img
        else:
          self.load_image()

    def load_image(self):

        if os.path.exists(self.img_path):
            self.image = cv2.imread(self.img_path)

    def read_image(self):

        if os.path.exists(self.img_path):
            return cv2.imread(self.img_path,cv2.IMREAD_UNCHANGED)
        else:
            raise FileNotFoundError(f"The file {self.img_path} does not exist.")

    def write_image(self, output_path):

        if self.image is not None:
            cv2.imwrite(output_path, self.image)
        else:
            raise ValueError("No image data available to write.")

    def to_grayscale(self):

        if self.image is not None:
            return cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        else:
            raise ValueError("No image data available for conversion.")

    def apply_threshold(self, threshold_value=128, max_value=255):

        gray_image = self.to_grayscale()
        _, thresh_image = cv2.threshold(gray_image, threshold_value, max_value, cv2.THRESH_BINARY)
        return thresh_image

    def resize(self, width, height):

        if self.image is not None:
            return cv2.resize(self.image, (width, height))
        else:
            raise ValueError("No image data available for resizing.")

    def show_image(self):

        if self.image is not None:
            cv2_imshow(self.resize(400,400))
        else:
            raise ValueError("No image data available to display.")

    def to_thresh(self):

        gray_image = self.to_grayscale()
        _, bw_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return bw_image

In [ ]:
import cv2
import numpy as np
from sklearn.metrics import jaccard_score


class Segment(Img):
    def __init__(self, img_path, mask, label, bbox, id=None,class_id=None):

        super().__init__(img_path)
        x_min = min([b[0] for b in bbox])
        y_min = min([b[1] for b in bbox])
        x_max = max([b[2] for b in bbox])
        y_max = max([b[3] for b in bbox])
        self.mask = mask
        self.label = label
        self.bbox = [x_min,y_min,x_max,y_max]
        self.id = id
        self.class_id = class_id

    def area(self):

        return np.sum(self.mask[:, :, 3]>0)

    def center(self):

        x_min, y_min, x_max, y_max = self.bbox
        return (x_min + x_max) / 2, (y_min + y_max) / 2

    @property
    def aspect_ratio(self):

        x_min, y_min, x_max, y_max = self.bbox
        width = x_max - x_min
        height = y_max - y_min
        return width / height if height != 0 else float('inf')  # Avoid division by zero

    @property
    def position(self):
        if self.image is None:
            self.image = self.load_image()
        img_height, img_width = self.image.shape[:2]
        x_center, y_center = self.center()
        # x_min, y_min, x_max, y_max = self.bbox
        return x_center / img_width, y_center / img_height

    def show_image(self):
        x_min, y_min, x_max, y_max = self.bbox
        cv2.rectangle(self.image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        return cv2_imshow(cv2.resize(self.image, (300,300), interpolation=cv2.INTER_LINEAR))



In [ ]:
def remove_hero_product(input_path):
  with open(input_path, "rb") as f:
      input_image = f.read()
  output_image = remove(input_image)

  image_without_bg = np.array(Image.open(io.BytesIO(output_image)))
  if image_without_bg.shape[-1] == 4:  # If image has 4 channels (RGBA)
    image_without_bg = cv2.cvtColor(image_without_bg, cv2.COLOR_RGBA2RGB)
  gray_image = cv2.cvtColor(image_without_bg, cv2.COLOR_BGR2GRAY)
  _, thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  bbox = []
  im2 = cv2.imread(input_path)
  for contour in contours:
      x, y, w, h = cv2.boundingRect(contour)
      if h < 30 or w<30:
        continue
      cv2.rectangle(image_without_bg, (x, y), (x + w, y + h), (0, 255, 0), 2)
      bbox.append([x,y,x+w,y+h])
      im2[y:y+h, x:x+w] = [255, 255, 255]
  im2 = cv2.resize(im2, (1000,1000), interpolation=cv2.INTER_AREA)
  image_name = input_path.split("/")[-1].split(".")[0]
  output_path = input_path.replace(image_name, image_name+"_hero_removed")
  cv2.imwrite(output_path, im2)
  return bbox,output_path

In [ ]:
import numpy as np
from skimage.measure import label

def merge_panoptic_segments(panoptic_seg, segments_info):
    """
    Merge adjacent segments with the same label in Mask2Former panoptic segmentation output.

    Args:
    - panoptic_seg: 2D array where each pixel value corresponds to a segment ID.
    - segments_info: List of dictionaries with segment details (from Mask2Former).
                     Each dictionary contains keys like 'id', 'category_id', etc.

    Returns:
    - merged_panoptic_seg: Updated 2D array with merged segment IDs.
    - merged_segments_info: Updated segment info with merged regions.
    """
    # Move panoptic_seg to CPU before creating a NumPy array
    panoptic_seg_cpu = panoptic_seg.cpu()  # Move to CPU

    # Create a new array for the merged segmentation using the CPU tensor
    merged_panoptic_seg = np.zeros_like(panoptic_seg_cpu, dtype=np.int32)
    current_segment_id = 1  # Start new segment IDs from 1
    merged_segments_info = []

    # Group segments by category_id
    category_to_segments = {}
    for segment in segments_info:
        category_id = segment['category_id']
        if category_id not in category_to_segments:
            category_to_segments[category_id] = []
        category_to_segments[category_id].append(segment['id'])

    # Process each category_id independently
    for category_id, segment_ids in category_to_segments.items():
        # Create a binary mask for all segments with the current category_id
        category_mask = np.isin(panoptic_seg_cpu, segment_ids).astype(np.uint8)

        # Find connected components within the mask
        labeled_components, num_components = label(category_mask, connectivity=2, return_num=True)

        for component_id in range(1, num_components + 1):
            merged_panoptic_seg[labeled_components == component_id] = current_segment_id
            merged_segments_info.append({
                "id": current_segment_id,
                "category_id": category_id,
                "area": np.sum(labeled_components == component_id)
            })
            current_segment_id += 1

    return merged_panoptic_seg, merged_segments_info

# Example usage:
# Assuming `outputs` is the result of Mask2Former inference
# panoptic_seg = outputs["panoptic_seg"][0].cpu().numpy()
# segments_info = outputs["panoptic_seg"][1]
# merged_panoptic_seg, merged_segments_info = merge_panoptic_segments(panoptic_seg, segments_info)

In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
import os
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
import cv2
import matplotlib.pyplot as plt

def remove_transparancy(image_path):
  image = Image.open(image_path)

  if image.mode in ('RGBA', 'LA') or (image.mode == 'P' and 'transparency' in image.info):
      white_bg = Image.new("RGB", image.size, (255, 255, 255))
      white_bg.paste(image, mask=image.split()[3])
      image = white_bg
      image.save(image_path)



def get_objects(img_path,output_base_dir="/content/seg/"):
  remove_transparancy(img_path)
  im = cv2.imread(img_path)
  img_name = img_path.split("/")[-1].split(".")[0]
  # cv2_imshow(im)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1], coco_metadata, scale=1.2, instance_mode=ColorMode.IMAGE_BW)
  panoptic_result = v.draw_panoptic_seg(outputs["panoptic_seg"][0].to("cpu"), outputs["panoptic_seg"][1]).get_image()
  # cv2_imshow(panoptic_result)

  panoptic_seg, segments_info = outputs["panoptic_seg"]
  # panoptic_seg, segments_info = merge_panoptic_segments(panoptic_seg, segments_info)
  panoptic_seg_np = panoptic_seg.to("cpu").numpy()
  # panoptic_seg_np = panoptic_seg


  im_with_boxes = im.copy()
  i=1
  segments = []
  class_name = " "
  for segment in segments_info:
      # print(i,segment,coco_metadata.thing_classes)
      i+=1
      segment_id = segment["id"]
      category_id = segment["category_id"]
      # category_name = coco_metadata.thing_classes[segment_id]
      for idx, class_name in enumerate(coco_metadata.stuff_classes):
          if category_id == idx:
            class_name = class_name.lower()
            break

      mask = (panoptic_seg_np == segment_id).astype(np.uint8)

      #%%%%%%%%%%%%%%%
      transparent_image = np.zeros((im.shape[0], im.shape[1], 4), dtype=np.uint8)
      transparent_image[..., :3] = im

      transparent_image_product = transparent_image.copy()
      transparent_image[..., 3] = ~np.where(mask, 255, 0)
      transparent_image_product[..., 3] = np.where(mask, 255, 0)
      #%%%%%%%%%%%%%%%%%%%%%%

      new_path = os.path.join(output_base_dir, img_name)
      os.makedirs(new_path, exist_ok=True)
      # print(new_path+"/"+str(segment_id)+".png")
      new_image_path = new_path+"/"+str(segment_id)+"_"+class_name+".png"
      cv2.imwrite(new_image_path, transparent_image_product)
      contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      bbox = []
      for contour in contours:
          x, y, w, h = cv2.boundingRect(contour)
          cv2.rectangle(im_with_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)
          bbox.append([x,y,x+w,y+h])
      segments.append(Segment(new_image_path ,transparent_image_product, class_name, bbox,segment_id))
  return segments

def add_bbox_to_image(src, bbox, color=(0, 255, 0), thickness=2):
    if isinstance(src, str):
        image = cv2.imread(src)
    else:
        image = src.copy()
    for x, y, x_max, y_max in bbox:
        cv2.rectangle(image, (x, y), (x_max, y_max), color, thickness)
    return image


def get_clip_embedding(image):
    if image.mode != 'RGB':
      image = image.convert('RGB')
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        embeddings = clip_model.get_image_features(**inputs)
    return embeddings / embeddings.norm(dim=-1, keepdim=True)  # Normalize

def crop_objects(image_path, regions):
    image = Image.open(image_path).convert("RGB")
    cropped_objects = []
    for region in regions:
        x_min, y_min, x_max, y_max = region
        cropped_object = image.crop((x_min, y_min, x_max, y_max))
        cropped_objects.append((cropped_object,(x_min, y_min, x_max, y_max)))
    return cropped_objects

def find_closest_object(target_image_path, target_regions, object_images_path, object_regions,ims_image,ims_silo):
    target_objects = crop_objects(target_image_path, target_regions)
    # print(target_objects)
    # for i, obj in enumerate(target_objects):
    #   obj[0].save(f"/content/{i}.png")
    target_embeddings = [(get_clip_embedding(obj[0]),obj[1]) for obj in target_objects]

    closest_image_path = None

    candidate_objects = crop_objects(object_images_path, object_regions)
    # for i, obj in enumerate(candidate_objects):
    #   obj[0].save(f"/content/{100+i}.png")
    candidate_embeddings = [(get_clip_embedding(obj[0]),obj[1]) for obj in candidate_objects]

    bbox = []
    final_imgs = []
    for candidate_embedding,img_silo in zip(candidate_embeddings,ims_silo):
        max_similarity = -1
        for target_embedding,img_image in zip(target_embeddings,ims_image):
            similarity = torch.cosine_similarity(target_embedding[0], candidate_embedding[0]).item()
            # print(similarity)
            if similarity > max_similarity:
                max_similarity = similarity
                # print(candidate_embedding[1],target_embedding[1])
                tar=target_embedding[1]
                bbox.append(tar)
                # final_imgs.append(img_image)
                cv2_imshow(img_image[0])
                cv2_imshow(img_image[1])

    return (min([b[0] for b in bbox]),min([b[1] for b in bbox]),max([b[2] for b in bbox]),max([b[3] for b in bbox])),final_imgs

def get_image_bbox(image_path,silo_path):
    image_path = resize_image(image_path, output_base_dir="/content/tmp/")
    silo_path = resize_image(silo_path, output_base_dir="/content/tmp/")
    target_regions,im1,ims_image = get_objects(image_path)
    object_regions,im3,ims_silo = get_objects(silo_path)
    bbox,imgs = find_closest_object(image_path, target_regions, silo_path, object_regions,ims_image,ims_silo)
    # cv2_imshow(im1)
    # import pdb;pdb.set_trace()
    # cv2_imshow(im3)
    # cv2_imshow(imgs[0][0])
    # cv2_imshow(imgs[0][1])

    return bbox

# bbox = get_image_bbox("/content/1.png","/content/silo.png")
# print(bbox)
# segments = get_objects("/content/6.png")
# get_objects("/content/2.png")
# get_objects("/content/4.png")
# get_objects("/content/5.png")
# get_objects("/content/3.png")
# get_objects("/content/6.png")
# image_path = "/content/tmp/AEDE2320/1.png"
# silo_path = "/content/test5/AEDE2320/silo.png"
# target_regions,im = get_objects(image_path)
# target_regions,im = detect_and_draw_bboxes(image_path)
# object_regions,im = get_objects(silo_path)
# object_regions,im = detect_and_draw_bboxes(silo_path)

# cv2_imshow(im)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [6]:

from rembg import remove

import cv2
import numpy as np
from PIL import Image
import io
def multi_scale_template_matching(img1, img2, scales=np.linspace(0.9, 1.1, 11), threshold=0.85):

    # Ensure img1 is the smaller image for cropping consistency
    if img1.shape[0] * img1.shape[1] > img2.shape[0] * img2.shape[1]:
        img1, img2 = img2, img1

    # Convert images to grayscale
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY) if len(img1.shape) == 3 else img1
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) if len(img2.shape) == 3 else img2

    # Normalize both images
    img1 = cv2.equalizeHist(img1)
    img2 = cv2.equalizeHist(img2)

    # Multi-scale template matching
    for scale in scales:
        # Resize the larger image
        scaled_img2 = cv2.resize(img2, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)

        # Template matching
        result = cv2.matchTemplate(scaled_img2, img1, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, _ = cv2.minMaxLoc(result)

        # Check if similarity exceeds the threshold
        if max_val >= threshold:
            return True

    return False

def are_images_similar(image1, image2, scales=np.linspace(0.9, 1.1, 11), threshold=0.85):

    if isinstance(image1, str):
        img1 = cv2.imread(image1)
    else:
        img1 = image1

    if isinstance(image2, str):
        img2 = cv2.imread(image2)
    else:
        img2 = image2



    return multi_scale_template_matching(img1, img2, scales, threshold)


def is_duplicate(image1, image2):
    _,output_path1=remove_hero_product(image1)
    _,output_path2=remove_hero_product(image2)
    seg_list1 = get_objects(output_path1)
    seg_list2 = get_objects(output_path2)
    print("*"*100)
    print(len(seg_list1),len(seg_list2))
    current_matches = []
    # import pdb;pdb.set_trace()
    for seg in seg_list1:
        get_best_match(seg, seg_list2, current_matches)
    not_matched_seg = []
    not_matched_seg2 = []

    if not current_matches:
      return False
    for s in seg_list1:
      if s not in [c[0] for c in current_matches]:
        not_matched_seg.append(s)

    for s in seg_list2:
      if s not in [c[1] for c in current_matches]:
        not_matched_seg2.append(s)
    import pdb;pdb.set_trace()

    if abs(len(current_matches)*2 - len(seg_list1) - len(seg_list2)) >= 6:
        return False
    # import pdb;pdb.set_trace()
    is_match_count = 0
    for matches in current_matches:
        is_same = are_images_similar(matches[0].image, matches[1].image)
        is_match_count += is_same
        # iobj = Img(img=matches[0].image)
        # iobj.resize(200,200)
        # iobj.show_image()
        # iobj2 = Img(img=matches[1].image)
        # iobj2.resize(200,200)
        # iobj2.show_image()
        # print(is_same)

    not_matching_image1_list = set(seg_list1) - set([m[0] for m in current_matches])
    not_matching_image2_list = set(seg_list2) - set([m[1] for m in current_matches])
    total_non_matching_area = sum(seg.area() for seg in not_matching_image1_list) + \
                              sum(seg.area() for seg in not_matching_image2_list)

    total_image_area = cv2.imread(image1).shape[0] * cv2.imread(image1).shape[1]
    max_not_matching_image_list = not_matching_image1_list
    min_not_matching_image_list = not_matching_image2_list
    if len(min_not_matching_image_list )>len(max_not_matching_image_list):
      max_not_matching_image_list = not_matching_image2_list
      min_not_matching_image_list = not_matching_image2_list


    is_match_count2 = 0
    import pdb;pdb.set_trace()

    for seg1 in max_not_matching_image_list:
      for seg2 in min_not_matching_image_list:
          is_same = are_images_similar(seg1.image, seg2.image)
          x1,y1,x2,y2 = seg1.bbox
          x3,y3,x4,y4 = seg2.bbox
          is_same = are_images_similar(seg1.image[y1:y2,x1:x2],seg2.image[y3:y4,x3:x4])

          is_match_count2 += is_same

    if is_match_count == len(current_matches):
        return True
    if total_non_matching_area < 0.03 * total_image_area:
        return True

    return False

def euclidean_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

def get_best_match(seg, seg_list2, current_matches):
    best_match = None
    best_score = float('inf')

    for candidate_seg in seg_list2:
        if candidate_seg in [m[1] for m in current_matches]:
            continue  # Skip already matched segments

        score = calculate_similarity(seg, candidate_seg)
        if score:
           current_matches.append((seg, candidate_seg))


    #     if score < best_score:
    #         best_score = score
    #         best_match = candidate_seg

    # if best_match is not None:
    #     current_matches.append((seg, best_match))


def get_products(image):
    image_resized = cv2.resize(image, (1000, 1000))

    segments = get_objects(image_resized)
    segments = [seg for seg in segments if seg.label != "wall"]

    hero_segments = [seg for seg in segments if is_hero_product(seg)]
    if len(hero_segments) > 1:
        hero_segments.sort(key=lambda seg: euclidean_distance(seg.center(), (500, 500)))
        hero_segments = hero_segments[:1]  # Keep only the closest one

    segments = [seg for seg in segments if seg not in hero_segments]

    return segments


def calculate_similarity(seg1, seg2):
    # - Aspect ratio
    aspect_ratio_diff = abs(seg1.aspect_ratio - seg2.aspect_ratio)<0.4
    max_aspect_ratio = max(seg1.aspect_ratio , seg2.aspect_ratio)
    if max_aspect_ratio>8:
           aspect_ratio_diff = abs(seg1.aspect_ratio - seg2.aspect_ratio)<2

    # - Position
    position_diff = abs(np.linalg.norm(np.array(seg1.position) -
                                   np.array(seg2.position)))<0.1

    # - Size (area difference)
    # import pdb;pdb.set_trace()
    size_diff = (seg1.area() - seg2.area())/(seg1.area()+ seg2.area())<0.1

    # - Category similarity
    category_diff = seg1.label == seg2.label

    0.4 * aspect_ratio_diff + 0.4 * position_diff + 0.2 * size_diff + category_diff
    # if seg1.label==seg2.label:
    #   print(seg1.label,seg2.label)
    #   seg1.show_image()
    #   seg2.show_image()

    #   print(abs(seg1.aspect_ratio - seg2.aspect_ratio))
    #   print(np.linalg.norm(np.array(seg1.position) -
    #                                np.array(seg2.position)))
    #   print((seg1.area() - seg2.area())/(seg1.area()+ seg2.area()))

    #   print(aspect_ratio_diff,position_diff,size_diff,category_diff)
      # import pdb;pdb.set_trace()
    return aspect_ratio_diff and position_diff and size_diff and category_diff


def is_hero_product(seg):

    return seg.area() > 0.1 * (1000 * 1000) and \
           euclidean_distance(seg.center(), (500, 500)) < 100



image1 = "/content/1.png"
image2 = "/content/2.png"
image3 = "/content/3.png"
image4 = "/content/4.png"
image5 = "/content/5.png"
image6 = "/content/6.png"
image06 = "/content/06.png"
image05 = "/content/05.png"
image01 = "/content/01.png"
image04 = "/content/04.png"
print(is_duplicate(image5, image2))
# for i in range(1,7):
#   for j in range(1,7):
#     i1 = f"/content/{i}.png"
#     i2 = f"/content/{j}.png"
# #     # print(i1,i2)
#     o = is_duplicate(i1, i2)
#     if o:
#       print(i1,i2)


NameError: name 'remove_hero_product' is not defined

In [ ]:
import glob

def is_dup_fname(fname1, fname2):
	fname1 = fname1.split(".")[0]
	base_name1 = fname1.split("-")[0]

	fname2 = fname2.split(".")[0]
	base_name2 = fname2.split("-")[0]

	if base_name1 == base_name2:
		return True

	return False


accuracy = 0
all_files = glob.glob("/content/*.png")

for fname1 in all_files:
	for fname2 in all_files:
		if fname1 == fname2:
			continue
		is_dup_flag = is_duplicate(fname1, fname2)
		ground_truth_flag = is_dup_fname(fname1, fname2)

		if is_dup_flag == ground_truth_flag:
			accuracy += 1

print("Accuracy = ", accuracy/len(all_files))


In [ ]:
from rembg import remove
import numpy as np
import cv2
from PIL import Image
import io

# Load the input image
input_path = "/content/2.png"  # Replace with your image file path
output_path = "/content/output_image.jpg"  # Output image path

# Step 1: Remove background using rembg
with open(input_path, "rb") as f:
    input_image = f.read()
output_image = remove(input_image)

# Convert the background-removed image to a format compatible with OpenCV
image_without_bg = np.array(Image.open(io.BytesIO(output_image)))

# Step 2: Convert to grayscale
gray_image = cv2.cvtColor(image_without_bg, cv2.COLOR_BGR2GRAY)

# Step 3: Apply binary thresholding
_, thresh = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)

# Step 4: Detect contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Step 5: Draw bounding boxes around detected contours
bbox = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if h < 30 or w<30:
      continue
    cv2.rectangle(image_without_bg, (x, y), (x + w, y + h), (0, 255, 0), 2)
    bbox.append([x,y,x+w,y+h])
print(bbox)

# Save the output image
cv2.imwrite(output_path, image_without_bg)
print(f"Output saved to {output_path}")

In [ ]:
!pip install onnxruntime

In [10]:
def remove_hero_product(input_path):
  with open(input_path, "rb") as f:
      input_image = f.read()
  output_image = remove(input_image)

  image_without_bg = np.array(Image.open(io.BytesIO(output_image)))
  if image_without_bg.shape[-1] == 4:  # If image has 4 channels (RGBA)
    image_without_bg = cv2.cvtColor(image_without_bg, cv2.COLOR_RGBA2RGB)
  gray_image = cv2.cvtColor(image_without_bg, cv2.COLOR_BGR2GRAY)
  _, thresh = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)
  contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  bbox = []
  im2 = cv2.imread(input_path)
  for contour in contours:
      x, y, w, h = cv2.boundingRect(contour)
      if h < 30 or w<30:
        continue
      cv2.rectangle(image_without_bg, (x, y), (x + w, y + h), (0, 255, 0), 2)
      bbox.append([x,y,x+w,y+h])
      im2[y:y+h, x:x+w] = [255, 255, 255]
  image_name = input_path.split("/")[-1].split(".")[0]
  output_path = input_path.replace(image_name, image_name+"_hero_removed")
  cv2.imwrite(output_path, im2)
  return bbox,output_path

# remove_hero_product("/content/6.png")

In [9]:
import cv2

def compare_sift(img1, img2):
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = [m for m, n in matches if m.distance < 0.55 * n.distance]
    return len(good_matches)


img1 = cv2.imread(f"/content/2-1.png")
img2 = cv2.imread(f"/content/2-1m.png")
s = compare_sift(img1, img2)
print(s)
# for i in range(1,7):
#   for j in range(1,7):
#     if i==j:
#       continue
#     img1 = cv2.imread(f"/content/{i}_hero_removed.png")
#     img2 = cv2.imread(f"/content/{j}_hero_removed.png")
#     s = compare_sift(img1, img2)

#     if s>40:
#       print(i,j,s)

4


In [11]:
import glob

def is_dup(im1,im2):
  _,im1_ = remove_hero_product(im1)
  _,im2_ = remove_hero_product(im2)

  img1 = cv2.imread(im1_)
  img2 = cv2.imread(im2_)
  s = compare_sift(img1, img2)

  if s>40:
    return True
  else:
    return False

def is_dup_fname(fname1, fname2):
	fname1 = fname1.split(".")[0]
	base_name1 = fname1.split("-")[0]

	fname2 = fname2.split(".")[0]
	base_name2 = fname2.split("-")[0]

	if base_name1 == base_name2:
		return True

	return False


accuracy = 0
all_files = glob.glob("/content/*.png")
all_files = [f for f in all_files if "hero_removed" not in f]
print(all_files)
for fname1 in all_files:
  for fname2 in all_files:
    if fname1 == fname2:
      continue
    print(fname1,fname2)
    is_dup_flag = is_dup(fname1, fname2)
    ground_truth_flag = is_dup_fname(fname1, fname2)
    print(is_dup_flag,ground_truth_flag)
#     if is_dup_flag == ground_truth_flag:
#       print(is_dup_flag,ground_truth_flag)
#       accuracy += 1

# print(accuracy)
# print("Accuracy = ", accuracy/len(all_files))

['/content/2-1m.png', '/content/2-1.png']
/content/2-1m.png /content/2-1.png


100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 175GB/s]


False True
/content/2-1.png /content/2-1m.png
False True


In [ ]:
import glob

def is_dup(im1,im2):
  _,im1_ = remove_hero_product(im1)
  _,im2_ = remove_hero_product(im2)

  img1 = cv2.imread(im1_)
  img2 = cv2.imread(im2_)
  try:
    s = compare_sift(img1, img2)
  except Exception as e:
    print("image failure")
    return False

  if s>40:
    return True
  else:
    return False

def is_dup_fname(fname1, fname2):
	fname1 = fname1.split(".")[0]
	base_name1 = fname1.split("-")[0]

	fname2 = fname2.split(".")[0]
	base_name2 = fname2.split("-")[0]

	if base_name1 == base_name2:
		return True

	return False


accuracy = 0
all_files = glob.glob("/content/dup3/*.png")
all_files = [f for f in all_files if "hero_removed" not in f]
print(all_files)
for fname1 in all_files:
  for fname2 in all_files:
    if fname1 == fname2:
      continue
    print(fname1,fname2)
    is_dup_flag = is_dup(fname1, fname2)
    ground_truth_flag = is_dup_fname(fname1, fname2)

    if is_dup_flag == ground_truth_flag:
      print(is_dup_flag,ground_truth_flag)
      accuracy += 1
    else:
      print("not match")

print(accuracy,len(all_files))
print("Accuracy = ", accuracy/len(all_files))

['/content/dup3/3-4.png', '/content/dup3/3-1.png', '/content/dup3/3-2.png', '/content/dup3/2-1.png', '/content/dup3/3-3.png', '/content/dup3/2.png', '/content/dup3/3.png']
/content/dup3/3-4.png /content/dup3/3-1.png
True True
/content/dup3/3-4.png /content/dup3/3-2.png
True True
/content/dup3/3-4.png /content/dup3/2-1.png
False False
/content/dup3/3-4.png /content/dup3/3-3.png
True True
/content/dup3/3-4.png /content/dup3/2.png
False False
/content/dup3/3-4.png /content/dup3/3.png
image failure
not match
/content/dup3/3-1.png /content/dup3/3-4.png
not match
/content/dup3/3-1.png /content/dup3/3-2.png
not match
/content/dup3/3-1.png /content/dup3/2-1.png
False False
/content/dup3/3-1.png /content/dup3/3-3.png
not match
/content/dup3/3-1.png /content/dup3/2.png
False False
/content/dup3/3-1.png /content/dup3/3.png
image failure
not match
/content/dup3/3-2.png /content/dup3/3-4.png
True True
/content/dup3/3-2.png /content/dup3/3-1.png
not match
/content/dup3/3-2.png /content/dup3/2-1.png


In [ ]:
import glob

def is_dup(im1,im2):
  _,im1_ = remove_hero_product(im1)
  _,im2_ = remove_hero_product(im2)

  img1 = cv2.imread(im1_)
  img2 = cv2.imread(im2_)
  try:
    s = compare_sift(img1, img2)
  except Exception as e:
    print("image failure")
    return False

  if s>40:
    return True
  else:
    return False

def is_dup_fname(fname1, fname2):
	fname1 = fname1.split(".")[0]
	base_name1 = fname1.split("-")[0]

	fname2 = fname2.split(".")[0]
	base_name2 = fname2.split("-")[0]

	if base_name1 == base_name2:
		return True

	return False


accuracy = 0
all_files = glob.glob("/content/dup4/*.png")
all_files = [f for f in all_files if "hero_removed" not in f]
print(all_files)
for fname1 in all_files:
  for fname2 in all_files:
    if fname1 == fname2:
      continue
    print(fname1,fname2)
    is_dup_flag = is_dup(fname1, fname2)
    ground_truth_flag = is_dup_fname(fname1, fname2)

    if is_dup_flag == ground_truth_flag:
      print(is_dup_flag,ground_truth_flag)
      accuracy += 1
    else:
      print("not match")

print(accuracy,len(all_files))
print("Accuracy = ", accuracy/len(all_files))

['/content/dup4/7.png', '/content/dup4/5-1.png', '/content/dup4/3-4.png', '/content/dup4/6-1.png', '/content/dup4/11-1.png', '/content/dup4/12.png', '/content/dup4/8-1.png', '/content/dup4/7-1.png', '/content/dup4/1-1.png', '/content/dup4/3-1.png', '/content/dup4/5.png', '/content/dup4/10-1.png', '/content/dup4/6.png', '/content/dup4/9-1.png', '/content/dup4/10.png', '/content/dup4/4-1.png', '/content/dup4/3-2.png', '/content/dup4/4.png', '/content/dup4/9.png', '/content/dup4/1.png', '/content/dup4/2-1.png', '/content/dup4/3-3.png', '/content/dup4/8.png', '/content/dup4/2.png', '/content/dup4/3.png', '/content/dup4/1-2.png', '/content/dup4/12-1.png', '/content/dup4/11.png']
/content/dup4/7.png /content/dup4/5-1.png
False False
/content/dup4/7.png /content/dup4/3-4.png
False False
/content/dup4/7.png /content/dup4/6-1.png
False False
/content/dup4/7.png /content/dup4/11-1.png
False False
/content/dup4/7.png /content/dup4/12.png
False False
/content/dup4/7.png /content/dup4/8-1.png
False

In [ ]:
/content/dup4/7.png /content/dup4/5-1.png
False False
/content/dup4/7.png /content/dup4/3-4.png
False False
/content/dup4/7.png /content/dup4/6-1.png
False False
/content/dup4/7.png /content/dup4/11-1.png
False False
/content/dup4/7.png /content/dup4/12.png
False False
/content/dup4/7.png /content/dup4/8-1.png
False False
/content/dup4/7.png /content/dup4/7-1.png
True True
/content/dup4/7.png /content/dup4/1-1.png
False False
/content/dup4/7.png /content/dup4/3-1.png
False False
/content/dup4/7.png /content/dup4/5.png
False False
/content/dup4/7.png /content/dup4/10-1.png
False False
/content/dup4/7.png /content/dup4/6.png
False False
/content/dup4/7.png /content/dup4/9-1.png
False False
/content/dup4/7.png /content/dup4/10.png
False False
/content/dup4/7.png /content/dup4/4-1.png
False False
/content/dup4/7.png /content/dup4/3-2.png
False False
/content/dup4/7.png /content/dup4/4.png
image failure
False False
/content/dup4/7.png /content/dup4/9.png
False False
/content/dup4/7.png /content/dup4/1.png
False False
/content/dup4/7.png /content/dup4/2-1.png
False False
/content/dup4/7.png /content/dup4/3-3.png
False False
/content/dup4/7.png /content/dup4/8.png
False False
/content/dup4/7.png /content/dup4/2.png
False False
/content/dup4/7.png /content/dup4/3.png
False False
/content/dup4/7.png /content/dup4/1-2.png
False False
/content/dup4/7.png /content/dup4/12-1.png
False False
/content/dup4/7.png /content/dup4/11.png
False False
/content/dup4/5-1.png /content/dup4/7.png
False False
/content/dup4/5-1.png /content/dup4/3-4.png
False False
/content/dup4/5-1.png /content/dup4/6-1.png
False False
/content/dup4/5-1.png /content/dup4/11-1.png
False False
/content/dup4/5-1.png /content/dup4/12.png
False False
/content/dup4/5-1.png /content/dup4/8-1.png
False False
/content/dup4/5-1.png /content/dup4/7-1.png
False False
/content/dup4/5-1.png /content/dup4/1-1.png
False False
/content/dup4/5-1.png /content/dup4/3-1.png
False False
/content/dup4/5-1.png /content/dup4/5.png
True True
/content/dup4/5-1.png /content/dup4/10-1.png
False False
/content/dup4/5-1.png /content/dup4/6.png
False False
/content/dup4/5-1.png /content/dup4/9-1.png
False False
/content/dup4/5-1.png /content/dup4/10.png
False False
/content/dup4/5-1.png /content/dup4/4-1.png
False False
/content/dup4/5-1.png /content/dup4/3-2.png
False False
/content/dup4/5-1.png /content/dup4/4.png
image failure
False False
/content/dup4/5-1.png /content/dup4/9.png
False False
/content/dup4/5-1.png /content/dup4/1.png
False False
/content/dup4/5-1.png /content/dup4/2-1.png
False False
/content/dup4/5-1.png /content/dup4/3-3.png
False False
/content/dup4/5-1.png /content/dup4/8.png
False False
/content/dup4/5-1.png /content/dup4/2.png
False False
/content/dup4/5-1.png /content/dup4/3.png
False False
/content/dup4/5-1.png /content/dup4/1-2.png
False False
/content/dup4/5-1.png /content/dup4/12-1.png
False False
/content/dup4/5-1.png /content/dup4/11.png
False False
/content/dup4/3-4.png /content/dup4/7.png
False False
/content/dup4/3-4.png /content/dup4/5-1.png
False False
/content/dup4/3-4.png /content/dup4/6-1.png
False False
/content/dup4/3-4.png /content/dup4/11-1.png
False False
/content/dup4/3-4.png /content/dup4/12.png
False False
/content/dup4/3-4.png /content/dup4/8-1.png
False False
/content/dup4/3-4.png /content/dup4/7-1.png
False False
/content/dup4/3-4.png /content/dup4/1-1.png
False False
/content/dup4/3-4.png /content/dup4/3-1.png
True True
/content/dup4/3-4.png /content/dup4/5.png
False False
/content/dup4/3-4.png /content/dup4/10-1.png
False False
/content/dup4/3-4.png /content/dup4/6.png
False False
/content/dup4/3-4.png /content/dup4/9-1.png
False False
/content/dup4/3-4.png /content/dup4/10.png
False False
/content/dup4/3-4.png /content/dup4/4-1.png
False False
/content/dup4/3-4.png /content/dup4/3-2.png
True True
/content/dup4/3-4.png /content/dup4/4.png
image failure
False False
/content/dup4/3-4.png /content/dup4/9.png
False False
/content/dup4/3-4.png /content/dup4/1.png
False False
/content/dup4/3-4.png /content/dup4/2-1.png
False False
/content/dup4/3-4.png /content/dup4/3-3.png
True True
/content/dup4/3-4.png /content/dup4/8.png
False False
/content/dup4/3-4.png /content/dup4/2.png
False False
/content/dup4/3-4.png /content/dup4/3.png
True True
/content/dup4/3-4.png /content/dup4/1-2.png
False False
/content/dup4/3-4.png /content/dup4/12-1.png
False False
/content/dup4/3-4.png /content/dup4/11.png
False False
/content/dup4/6-1.png /content/dup4/7.png
False False
/content/dup4/6-1.png /content/dup4/5-1.png
False False
/content/dup4/6-1.png /content/dup4/3-4.png
False False
/content/dup4/6-1.png /content/dup4/11-1.png
False False
/content/dup4/6-1.png /content/dup4/12.png
False False
/content/dup4/6-1.png /content/dup4/8-1.png
False False
/content/dup4/6-1.png /content/dup4/7-1.png
False False
/content/dup4/6-1.png /content/dup4/1-1.png
False False
/content/dup4/6-1.png /content/dup4/3-1.png
False False
/content/dup4/6-1.png /content/dup4/5.png
False False
/content/dup4/6-1.png /content/dup4/10-1.png
False False
/content/dup4/6-1.png /content/dup4/6.png
True True
/content/dup4/6-1.png /content/dup4/9-1.png
False False
/content/dup4/6-1.png /content/dup4/10.png
False False
/content/dup4/6-1.png /content/dup4/4-1.png
False False
/content/dup4/6-1.png /content/dup4/3-2.png
False False
/content/dup4/6-1.png /content/dup4/4.png
image failure
False False
/content/dup4/6-1.png /content/dup4/9.png
False False
/content/dup4/6-1.png /content/dup4/1.png
False False
/content/dup4/6-1.png /content/dup4/2-1.png
False False
/content/dup4/6-1.png /content/dup4/3-3.png
False False
/content/dup4/6-1.png /content/dup4/8.png
False False
/content/dup4/6-1.png /content/dup4/2.png
False False
/content/dup4/6-1.png /content/dup4/3.png
False False
/content/dup4/6-1.png /content/dup4/1-2.png
False False
/content/dup4/6-1.png /content/dup4/12-1.png
False False
/content/dup4/6-1.png /content/dup4/11.png
False False
/content/dup4/11-1.png /content/dup4/7.png
False False
/content/dup4/11-1.png /content/dup4/5-1.png
False False
/content/dup4/11-1.png /content/dup4/3-4.png
False False
/content/dup4/11-1.png /content/dup4/6-1.png
False False
/content/dup4/11-1.png /content/dup4/12.png
False False
/content/dup4/11-1.png /content/dup4/8-1.png
False False
/content/dup4/11-1.png /content/dup4/7-1.png
False False
/content/dup4/11-1.png /content/dup4/1-1.png
False False
/content/dup4/11-1.png /content/dup4/3-1.png
False False
/content/dup4/11-1.png /content/dup4/5.png
False False
/content/dup4/11-1.png /content/dup4/10-1.png
False False
/content/dup4/11-1.png /content/dup4/6.png
False False
/content/dup4/11-1.png /content/dup4/9-1.png
False False
/content/dup4/11-1.png /content/dup4/10.png
False False
/content/dup4/11-1.png /content/dup4/4-1.png
False False
/content/dup4/11-1.png /content/dup4/3-2.png
False False
/content/dup4/11-1.png /content/dup4/4.png
image failure
False False
/content/dup4/11-1.png /content/dup4/9.png
False False
/content/dup4/11-1.png /content/dup4/1.png
False False
/content/dup4/11-1.png /content/dup4/2-1.png
False False
/content/dup4/11-1.png /content/dup4/3-3.png
False False
/content/dup4/11-1.png /content/dup4/8.png
False False
/content/dup4/11-1.png /content/dup4/2.png
False False
/content/dup4/11-1.png /content/dup4/3.png
False False
/content/dup4/11-1.png /content/dup4/1-2.png
False False
/content/dup4/11-1.png /content/dup4/12-1.png
False False
/content/dup4/11-1.png /content/dup4/11.png
True True
/content/dup4/12.png /content/dup4/7.png
False False
/content/dup4/12.png /content/dup4/5-1.png
False False
/content/dup4/12.png /content/dup4/3-4.png
False False
/content/dup4/12.png /content/dup4/6-1.png
False False
/content/dup4/12.png /content/dup4/11-1.png
False False
/content/dup4/12.png /content/dup4/8-1.png
False False
/content/dup4/12.png /content/dup4/7-1.png
False False
/content/dup4/12.png /content/dup4/1-1.png
False False
/content/dup4/12.png /content/dup4/3-1.png
False False
/content/dup4/12.png /content/dup4/5.png
False False
/content/dup4/12.png /content/dup4/10-1.png
False False
/content/dup4/12.png /content/dup4/6.png
False False
/content/dup4/12.png /content/dup4/9-1.png
False False
/content/dup4/12.png /content/dup4/10.png
False False
/content/dup4/12.png /content/dup4/4-1.png
False False
/content/dup4/12.png /content/dup4/3-2.png
False False
/content/dup4/12.png /content/dup4/4.png
image failure
False False
/content/dup4/12.png /content/dup4/9.png
False False
/content/dup4/12.png /content/dup4/1.png
False False
/content/dup4/12.png /content/dup4/2-1.png
False False
/content/dup4/12.png /content/dup4/3-3.png
False False
/content/dup4/12.png /content/dup4/8.png
False False
/content/dup4/12.png /content/dup4/2.png
False False
/content/dup4/12.png /content/dup4/3.png
False False
/content/dup4/12.png /content/dup4/1-2.png
False False
/content/dup4/12.png /content/dup4/12-1.png
True True
/content/dup4/12.png /content/dup4/11.png
False False
/content/dup4/8-1.png /content/dup4/7.png
False False
/content/dup4/8-1.png /content/dup4/5-1.png
False False
/content/dup4/8-1.png /content/dup4/3-4.png
False False
/content/dup4/8-1.png /content/dup4/6-1.png
False False
/content/dup4/8-1.png /content/dup4/11-1.png
False False
/content/dup4/8-1.png /content/dup4/12.png
False False
/content/dup4/8-1.png /content/dup4/7-1.png
False False
/content/dup4/8-1.png /content/dup4/1-1.png
False False
/content/dup4/8-1.png /content/dup4/3-1.png
False False
/content/dup4/8-1.png /content/dup4/5.png
False False
/content/dup4/8-1.png /content/dup4/10-1.png
False False
/content/dup4/8-1.png /content/dup4/6.png
False False
/content/dup4/8-1.png /content/dup4/9-1.png
False False
/content/dup4/8-1.png /content/dup4/10.png
False False
/content/dup4/8-1.png /content/dup4/4-1.png
False False
/content/dup4/8-1.png /content/dup4/3-2.png
False False
/content/dup4/8-1.png /content/dup4/4.png
image failure
False False
/content/dup4/8-1.png /content/dup4/9.png
False False
/content/dup4/8-1.png /content/dup4/1.png
False False
/content/dup4/8-1.png /content/dup4/2-1.png
False False
/content/dup4/8-1.png /content/dup4/3-3.png
False False
/content/dup4/8-1.png /content/dup4/8.png
not match
/content/dup4/8-1.png /content/dup4/2.png
False False
/content/dup4/8-1.png /content/dup4/3.png
False False
/content/dup4/8-1.png /content/dup4/1-2.png
False False
/content/dup4/8-1.png /content/dup4/12-1.png
False False
/content/dup4/8-1.png /content/dup4/11.png
False False
/content/dup4/7-1.png /content/dup4/7.png
True True
/content/dup4/7-1.png /content/dup4/5-1.png
False False
/content/dup4/7-1.png /content/dup4/3-4.png
False False
/content/dup4/7-1.png /content/dup4/6-1.png
False False
/content/dup4/7-1.png /content/dup4/11-1.png
False False
/content/dup4/7-1.png /content/dup4/12.png
False False
/content/dup4/7-1.png /content/dup4/8-1.png
False False
/content/dup4/7-1.png /content/dup4/1-1.png
False False
/content/dup4/7-1.png /content/dup4/3-1.png
False False
/content/dup4/7-1.png /content/dup4/5.png
False False
/content/dup4/7-1.png /content/dup4/10-1.png
False False
/content/dup4/7-1.png /content/dup4/6.png
False False
/content/dup4/7-1.png /content/dup4/9-1.png
False False
/content/dup4/7-1.png /content/dup4/10.png
False False
/content/dup4/7-1.png /content/dup4/4-1.png
False False
/content/dup4/7-1.png /content/dup4/3-2.png
False False
/content/dup4/7-1.png /content/dup4/4.png
image failure
False False
/content/dup4/7-1.png /content/dup4/9.png
False False
/content/dup4/7-1.png /content/dup4/1.png
False False
/content/dup4/7-1.png /content/dup4/2-1.png
False False
/content/dup4/7-1.png /content/dup4/3-3.png
False False
/content/dup4/7-1.png /content/dup4/8.png
False False
/content/dup4/7-1.png /content/dup4/2.png
False False
/content/dup4/7-1.png /content/dup4/3.png
False False
/content/dup4/7-1.png /content/dup4/1-2.png
False False
/content/dup4/7-1.png /content/dup4/12-1.png
False False
/content/dup4/7-1.png /content/dup4/11.png
False False
/content/dup4/1-1.png /content/dup4/7.png
False False
/content/dup4/1-1.png /content/dup4/5-1.png
False False
/content/dup4/1-1.png /content/dup4/3-4.png
False False
/content/dup4/1-1.png /content/dup4/6-1.png
False False
/content/dup4/1-1.png /content/dup4/11-1.png
False False
/content/dup4/1-1.png /content/dup4/12.png
False False
/content/dup4/1-1.png /content/dup4/8-1.png
False False
/content/dup4/1-1.png /content/dup4/7-1.png
False False
/content/dup4/1-1.png /content/dup4/3-1.png
False False
/content/dup4/1-1.png /content/dup4/5.png
False False
/content/dup4/1-1.png /content/dup4/10-1.png
False False
/content/dup4/1-1.png /content/dup4/6.png
False False
/content/dup4/1-1.png /content/dup4/9-1.png
False False
/content/dup4/1-1.png /content/dup4/10.png
False False
/content/dup4/1-1.png /content/dup4/4-1.png
False False
/content/dup4/1-1.png /content/dup4/3-2.png
False False
/content/dup4/1-1.png /content/dup4/4.png
image failure
False False
/content/dup4/1-1.png /content/dup4/9.png
False False
/content/dup4/1-1.png /content/dup4/1.png
True True
/content/dup4/1-1.png /content/dup4/2-1.png
False False
/content/dup4/1-1.png /content/dup4/3-3.png
False False
/content/dup4/1-1.png /content/dup4/8.png
False False
/content/dup4/1-1.png /content/dup4/2.png
False False
/content/dup4/1-1.png /content/dup4/3.png
False False
/content/dup4/1-1.png /content/dup4/1-2.png
True True
/content/dup4/1-1.png /content/dup4/12-1.png
False False
/content/dup4/1-1.png /content/dup4/11.png
False False
/content/dup4/3-1.png /content/dup4/7.png
False False
/content/dup4/3-1.png /content/dup4/5-1.png
False False
/content/dup4/3-1.png /content/dup4/3-4.png
not match
/content/dup4/3-1.png /content/dup4/6-1.png
False False
/content/dup4/3-1.png /content/dup4/11-1.png
False False
/content/dup4/3-1.png /content/dup4/12.png
False False
/content/dup4/3-1.png /content/dup4/8-1.png
False False
/content/dup4/3-1.png /content/dup4/7-1.png
False False
/content/dup4/3-1.png /content/dup4/1-1.png
False False
/content/dup4/3-1.png /content/dup4/5.png
False False
/content/dup4/3-1.png /content/dup4/10-1.png
False False
/content/dup4/3-1.png /content/dup4/6.png
False False
/content/dup4/3-1.png /content/dup4/9-1.png
False False
/content/dup4/3-1.png /content/dup4/10.png
False False
/content/dup4/3-1.png /content/dup4/4-1.png
False False
/content/dup4/3-1.png /content/dup4/3-2.png
not match
/content/dup4/3-1.png /content/dup4/4.png
image failure
False False
/content/dup4/3-1.png /content/dup4/9.png
False False
/content/dup4/3-1.png /content/dup4/1.png
False False
/content/dup4/3-1.png /content/dup4/2-1.png
False False
/content/dup4/3-1.png /content/dup4/3-3.png
not match
/content/dup4/3-1.png /content/dup4/8.png
False False
/content/dup4/3-1.png /content/dup4/2.png
False False
/content/dup4/3-1.png /content/dup4/3.png
True True
/content/dup4/3-1.png /content/dup4/1-2.png
False False
/content/dup4/3-1.png /content/dup4/12-1.png
False False
/content/dup4/3-1.png /content/dup4/11.png
False False
/content/dup4/5.png /content/dup4/7.png
False False
/content/dup4/5.png /content/dup4/5-1.png
True True
/content/dup4/5.png /content/dup4/3-4.png
False False
/content/dup4/5.png /content/dup4/6-1.png
False False
/content/dup4/5.png /content/dup4/11-1.png
False False
/content/dup4/5.png /content/dup4/12.png
False False
/content/dup4/5.png /content/dup4/8-1.png
False False
/content/dup4/5.png /content/dup4/7-1.png
False False
/content/dup4/5.png /content/dup4/1-1.png
False False
/content/dup4/5.png /content/dup4/3-1.png
False False
/content/dup4/5.png /content/dup4/10-1.png
False False
/content/dup4/5.png /content/dup4/6.png
False False
/content/dup4/5.png /content/dup4/9-1.png
False False
/content/dup4/5.png /content/dup4/10.png
False False
/content/dup4/5.png /content/dup4/4-1.png
False False
/content/dup4/5.png /content/dup4/3-2.png
False False
/content/dup4/5.png /content/dup4/4.png
image failure
False False
/content/dup4/5.png /content/dup4/9.png
False False
/content/dup4/5.png /content/dup4/1.png
False False
/content/dup4/5.png /content/dup4/2-1.png
False False
/content/dup4/5.png /content/dup4/3-3.png
False False
/content/dup4/5.png /content/dup4/8.png
False False
/content/dup4/5.png /content/dup4/2.png
False False
/content/dup4/5.png /content/dup4/3.png
False False
/content/dup4/5.png /content/dup4/1-2.png
False False
/content/dup4/5.png /content/dup4/12-1.png
False False
/content/dup4/5.png /content/dup4/11.png
False False
/content/dup4/10-1.png /content/dup4/7.png
False False
/content/dup4/10-1.png /content/dup4/5-1.png
False False
/content/dup4/10-1.png /content/dup4/3-4.png
False False
/content/dup4/10-1.png /content/dup4/6-1.png
False False
/content/dup4/10-1.png /content/dup4/11-1.png
False False
/content/dup4/10-1.png /content/dup4/12.png
False False
/content/dup4/10-1.png /content/dup4/8-1.png
False False
/content/dup4/10-1.png /content/dup4/7-1.png
False False
/content/dup4/10-1.png /content/dup4/1-1.png
False False
/content/dup4/10-1.png /content/dup4/3-1.png
False False
/content/dup4/10-1.png /content/dup4/5.png
False False
/content/dup4/10-1.png /content/dup4/6.png
False False
/content/dup4/10-1.png /content/dup4/9-1.png
False False
/content/dup4/10-1.png /content/dup4/10.png
True True
/content/dup4/10-1.png /content/dup4/4-1.png
False False
/content/dup4/10-1.png /content/dup4/3-2.png
False False
/content/dup4/10-1.png /content/dup4/4.png
image failure
False False
/content/dup4/10-1.png /content/dup4/9.png
False False
/content/dup4/10-1.png /content/dup4/1.png
False False
/content/dup4/10-1.png /content/dup4/2-1.png
False False
/content/dup4/10-1.png /content/dup4/3-3.png
False False
/content/dup4/10-1.png /content/dup4/8.png
False False
/content/dup4/10-1.png /content/dup4/2.png
False False
/content/dup4/10-1.png /content/dup4/3.png
False False
/content/dup4/10-1.png /content/dup4/1-2.png
False False
/content/dup4/10-1.png /content/dup4/12-1.png
False False
/content/dup4/10-1.png /content/dup4/11.png
False False
/content/dup4/6.png /content/dup4/7.png
False False
/content/dup4/6.png /content/dup4/5-1.png
False False
/content/dup4/6.png /content/dup4/3-4.png
False False
/content/dup4/6.png /content/dup4/6-1.png
True True
/content/dup4/6.png /content/dup4/11-1.png
False False
/content/dup4/6.png /content/dup4/12.png
False False
/content/dup4/6.png /content/dup4/8-1.png
False False
/content/dup4/6.png /content/dup4/7-1.png
False False
/content/dup4/6.png /content/dup4/1-1.png
False False
/content/dup4/6.png /content/dup4/3-1.png
False False
/content/dup4/6.png /content/dup4/5.png
False False
/content/dup4/6.png /content/dup4/10-1.png
False False
/content/dup4/6.png /content/dup4/9-1.png
False False
/content/dup4/6.png /content/dup4/10.png
False False
/content/dup4/6.png /content/dup4/4-1.png
False False
/content/dup4/6.png /content/dup4/3-2.png
False False
/content/dup4/6.png /content/dup4/4.png
image failure
False False
/content/dup4/6.png /content/dup4/9.png
False False
/content/dup4/6.png /content/dup4/1.png
False False
/content/dup4/6.png /content/dup4/2-1.png
False False
/content/dup4/6.png /content/dup4/3-3.png
False False
/content/dup4/6.png /content/dup4/8.png
False False
/content/dup4/6.png /content/dup4/2.png
False False
/content/dup4/6.png /content/dup4/3.png
False False
/content/dup4/6.png /content/dup4/1-2.png
False False
/content/dup4/6.png /content/dup4/12-1.png
False False
/content/dup4/6.png /content/dup4/11.png
False False
/content/dup4/9-1.png /content/dup4/7.png
False False
/content/dup4/9-1.png /content/dup4/5-1.png
False False
/content/dup4/9-1.png /content/dup4/3-4.png
False False
/content/dup4/9-1.png /content/dup4/6-1.png
False False
/content/dup4/9-1.png /content/dup4/11-1.png
False False
/content/dup4/9-1.png /content/dup4/12.png
False False
/content/dup4/9-1.png /content/dup4/8-1.png
False False
/content/dup4/9-1.png /content/dup4/7-1.png
False False
/content/dup4/9-1.png /content/dup4/1-1.png
False False
/content/dup4/9-1.png /content/dup4/3-1.png
False False
/content/dup4/9-1.png /content/dup4/5.png
False False
/content/dup4/9-1.png /content/dup4/10-1.png
False False
/content/dup4/9-1.png /content/dup4/6.png
False False
/content/dup4/9-1.png /content/dup4/10.png
False False
/content/dup4/9-1.png /content/dup4/4-1.png
False False
/content/dup4/9-1.png /content/dup4/3-2.png
False False
/content/dup4/9-1.png /content/dup4/4.png
image failure
False False
/content/dup4/9-1.png /content/dup4/9.png
not match
/content/dup4/9-1.png /content/dup4/1.png
False False
/content/dup4/9-1.png /content/dup4/2-1.png
False False
/content/dup4/9-1.png /content/dup4/3-3.png
False False
/content/dup4/9-1.png /content/dup4/8.png
False False
/content/dup4/9-1.png /content/dup4/2.png
False False
/content/dup4/9-1.png /content/dup4/3.png
False False
/content/dup4/9-1.png /content/dup4/1-2.png
False False
/content/dup4/9-1.png /content/dup4/12-1.png
False False
/content/dup4/9-1.png /content/dup4/11.png
False False
/content/dup4/10.png /content/dup4/7.png
False False
/content/dup4/10.png /content/dup4/5-1.png
False False
/content/dup4/10.png /content/dup4/3-4.png
False False
/content/dup4/10.png /content/dup4/6-1.png
False False
/content/dup4/10.png /content/dup4/11-1.png
False False
/content/dup4/10.png /content/dup4/12.png
False False
/content/dup4/10.png /content/dup4/8-1.png
False False
/content/dup4/10.png /content/dup4/7-1.png
False False
/content/dup4/10.png /content/dup4/1-1.png
False False
/content/dup4/10.png /content/dup4/3-1.png
False False
/content/dup4/10.png /content/dup4/5.png
False False
/content/dup4/10.png /content/dup4/10-1.png
True True
/content/dup4/10.png /content/dup4/6.png
False False
/content/dup4/10.png /content/dup4/9-1.png
False False
/content/dup4/10.png /content/dup4/4-1.png
False False
/content/dup4/10.png /content/dup4/3-2.png
False False
/content/dup4/10.png /content/dup4/4.png
image failure
False False
/content/dup4/10.png /content/dup4/9.png
False False
/content/dup4/10.png /content/dup4/1.png
False False
/content/dup4/10.png /content/dup4/2-1.png
False False
/content/dup4/10.png /content/dup4/3-3.png
False False
/content/dup4/10.png /content/dup4/8.png
False False
/content/dup4/10.png /content/dup4/2.png
False False
/content/dup4/10.png /content/dup4/3.png
False False
/content/dup4/10.png /content/dup4/1-2.png
False False
/content/dup4/10.png /content/dup4/12-1.png
False False
/content/dup4/10.png /content/dup4/11.png
False False
/content/dup4/4-1.png /content/dup4/7.png
False False
/content/dup4/4-1.png /content/dup4/5-1.png
False False
/content/dup4/4-1.png /content/dup4/3-4.png
False False
/content/dup4/4-1.png /content/dup4/6-1.png
False False
/content/dup4/4-1.png /content/dup4/11-1.png
False False
/content/dup4/4-1.png /content/dup4/12.png
False False
/content/dup4/4-1.png /content/dup4/8-1.png
False False
/content/dup4/4-1.png /content/dup4/7-1.png
False False
/content/dup4/4-1.png /content/dup4/1-1.png
False False
/content/dup4/4-1.png /content/dup4/3-1.png
False False
/content/dup4/4-1.png /content/dup4/5.png
False False
/content/dup4/4-1.png /content/dup4/10-1.png
False False
/content/dup4/4-1.png /content/dup4/6.png
False False
/content/dup4/4-1.png /content/dup4/9-1.png
False False
/content/dup4/4-1.png /content/dup4/10.png
False False
/content/dup4/4-1.png /content/dup4/3-2.png
False False
/content/dup4/4-1.png /content/dup4/4.png
image failure
not match
/content/dup4/4-1.png /content/dup4/9.png
False False
/content/dup4/4-1.png /content/dup4/1.png
False False
/content/dup4/4-1.png /content/dup4/2-1.png
False False
/content/dup4/4-1.png /content/dup4/3-3.png
False False
/content/dup4/4-1.png /content/dup4/8.png
False False
/content/dup4/4-1.png /content/dup4/2.png
False False
/content/dup4/4-1.png /content/dup4/3.png
False False
/content/dup4/4-1.png /content/dup4/1-2.png
False False
/content/dup4/4-1.png /content/dup4/12-1.png
False False
/content/dup4/4-1.png /content/dup4/11.png
False False
/content/dup4/3-2.png /content/dup4/7.png
False False
/content/dup4/3-2.png /content/dup4/5-1.png
False False
/content/dup4/3-2.png /content/dup4/3-4.png
True True
/content/dup4/3-2.png /content/dup4/6-1.png
False False
/content/dup4/3-2.png /content/dup4/11-1.png
False False
/content/dup4/3-2.png /content/dup4/12.png
False False
/content/dup4/3-2.png /content/dup4/8-1.png
False False
/content/dup4/3-2.png /content/dup4/7-1.png
False False
/content/dup4/3-2.png /content/dup4/1-1.png
False False
/content/dup4/3-2.png /content/dup4/3-1.png
not match
/content/dup4/3-2.png /content/dup4/5.png
False False
/content/dup4/3-2.png /content/dup4/10-1.png
False False
/content/dup4/3-2.png /content/dup4/6.png
False False
/content/dup4/3-2.png /content/dup4/9-1.png
False False
/content/dup4/3-2.png /content/dup4/10.png
False False
/content/dup4/3-2.png /content/dup4/4-1.png
False False
/content/dup4/3-2.png /content/dup4/4.png
image failure
False False
/content/dup4/3-2.png /content/dup4/9.png
False False
/content/dup4/3-2.png /content/dup4/1.png
False False
/content/dup4/3-2.png /content/dup4/2-1.png
False False
/content/dup4/3-2.png /content/dup4/3-3.png
True True
/content/dup4/3-2.png /content/dup4/8.png
False False
/content/dup4/3-2.png /content/dup4/2.png
False False
/content/dup4/3-2.png /content/dup4/3.png
True True
/content/dup4/3-2.png /content/dup4/1-2.png
False False
/content/dup4/3-2.png /content/dup4/12-1.png
False False
/content/dup4/3-2.png /content/dup4/11.png
False False
/content/dup4/4.png /content/dup4/7.png
image failure
False False
/content/dup4/4.png /content/dup4/5-1.png
image failure
False False
/content/dup4/4.png /content/dup4/3-4.png
image failure
False False
/content/dup4/4.png /content/dup4/6-1.png
image failure
False False
/content/dup4/4.png /content/dup4/11-1.png
image failure
False False
/content/dup4/4.png /content/dup4/12.png
image failure
False False
/content/dup4/4.png /content/dup4/8-1.png
image failure
False False
/content/dup4/4.png /content/dup4/7-1.png
image failure
False False
/content/dup4/4.png /content/dup4/1-1.png
image failure
False False
/content/dup4/4.png /content/dup4/3-1.png
image failure
False False
/content/dup4/4.png /content/dup4/5.png
image failure
False False
/content/dup4/4.png /content/dup4/10-1.png
image failure
False False
/content/dup4/4.png /content/dup4/6.png
image failure
False False
/content/dup4/4.png /content/dup4/9-1.png
image failure
False False
/content/dup4/4.png /content/dup4/10.png
image failure
False False
/content/dup4/4.png /content/dup4/4-1.png
image failure
not match
/content/dup4/4.png /content/dup4/3-2.png
image failure
False False
/content/dup4/4.png /content/dup4/9.png
image failure
False False
/content/dup4/4.png /content/dup4/1.png
image failure
False False
/content/dup4/4.png /content/dup4/2-1.png
image failure
False False
/content/dup4/4.png /content/dup4/3-3.png
image failure
False False
/content/dup4/4.png /content/dup4/8.png
image failure
False False
/content/dup4/4.png /content/dup4/2.png
image failure
False False
/content/dup4/4.png /content/dup4/3.png
image failure
False False
/content/dup4/4.png /content/dup4/1-2.png
image failure
False False
/content/dup4/4.png /content/dup4/12-1.png
image failure
False False
/content/dup4/4.png /content/dup4/11.png
image failure
False False
/content/dup4/9.png /content/dup4/7.png
False False
/content/dup4/9.png /content/dup4/5-1.png
False False
/content/dup4/9.png /content/dup4/3-4.png
False False
/content/dup4/9.png /content/dup4/6-1.png
False False
/content/dup4/9.png /content/dup4/11-1.png
False False
/content/dup4/9.png /content/dup4/12.png
False False
/content/dup4/9.png /content/dup4/8-1.png
False False
/content/dup4/9.png /content/dup4/7-1.png
False False
/content/dup4/9.png /content/dup4/1-1.png
False False
/content/dup4/9.png /content/dup4/3-1.png
False False
/content/dup4/9.png /content/dup4/5.png
False False
/content/dup4/9.png /content/dup4/10-1.png
False False
/content/dup4/9.png /content/dup4/6.png
False False
/content/dup4/9.png /content/dup4/9-1.png
not match
/content/dup4/9.png /content/dup4/10.png
False False
/content/dup4/9.png /content/dup4/4-1.png
False False
/content/dup4/9.png /content/dup4/3-2.png
False False
/content/dup4/9.png /content/dup4/4.png
image failure
False False
/content/dup4/9.png /content/dup4/1.png
False False
/content/dup4/9.png /content/dup4/2-1.png
False False
/content/dup4/9.png /content/dup4/3-3.png
False False
/content/dup4/9.png /content/dup4/8.png
False False
/content/dup4/9.png /content/dup4/2.png
False False
/content/dup4/9.png /content/dup4/3.png
False False
/content/dup4/9.png /content/dup4/1-2.png
False False
/content/dup4/9.png /content/dup4/12-1.png
False False
/content/dup4/9.png /content/dup4/11.png
False False
/content/dup4/1.png /content/dup4/7.png
False False
/content/dup4/1.png /content/dup4/5-1.png
False False
/content/dup4/1.png /content/dup4/3-4.png
False False
/content/dup4/1.png /content/dup4/6-1.png
False False
/content/dup4/1.png /content/dup4/11-1.png
False False
/content/dup4/1.png /content/dup4/12.png
False False
/content/dup4/1.png /content/dup4/8-1.png
False False
/content/dup4/1.png /content/dup4/7-1.png
False False
/content/dup4/1.png /content/dup4/1-1.png
True True
/content/dup4/1.png /content/dup4/3-1.png
False False
/content/dup4/1.png /content/dup4/5.png
False False
/content/dup4/1.png /content/dup4/10-1.png
False False
/content/dup4/1.png /content/dup4/6.png
False False
/content/dup4/1.png /content/dup4/9-1.png
False False
/content/dup4/1.png /content/dup4/10.png
False False
/content/dup4/1.png /content/dup4/4-1.png
False False
/content/dup4/1.png /content/dup4/3-2.png
False False
/content/dup4/1.png /content/dup4/4.png
image failure
False False
/content/dup4/1.png /content/dup4/9.png
False False
/content/dup4/1.png /content/dup4/2-1.png
False False
/content/dup4/1.png /content/dup4/3-3.png
False False
/content/dup4/1.png /content/dup4/8.png
False False
/content/dup4/1.png /content/dup4/2.png
False False
/content/dup4/1.png /content/dup4/3.png
False False
/content/dup4/1.png /content/dup4/1-2.png
True True
/content/dup4/1.png /content/dup4/12-1.png
False False
/content/dup4/1.png /content/dup4/11.png
False False
/content/dup4/2-1.png /content/dup4/7.png
False False
/content/dup4/2-1.png /content/dup4/5-1.png
False False
/content/dup4/2-1.png /content/dup4/3-4.png
False False
/content/dup4/2-1.png /content/dup4/6-1.png
False False
/content/dup4/2-1.png /content/dup4/11-1.png
False False
/content/dup4/2-1.png /content/dup4/12.png
False False
/content/dup4/2-1.png /content/dup4/8-1.png
False False
/content/dup4/2-1.png /content/dup4/7-1.png
False False
/content/dup4/2-1.png /content/dup4/1-1.png
False False
/content/dup4/2-1.png /content/dup4/3-1.png
False False
/content/dup4/2-1.png /content/dup4/5.png
False False
/content/dup4/2-1.png /content/dup4/10-1.png
False False
/content/dup4/2-1.png /content/dup4/6.png
False False
/content/dup4/2-1.png /content/dup4/9-1.png
False False
/content/dup4/2-1.png /content/dup4/10.png
False False
/content/dup4/2-1.png /content/dup4/4-1.png
False False
/content/dup4/2-1.png /content/dup4/3-2.png
False False
/content/dup4/2-1.png /content/dup4/4.png
image failure
False False
/content/dup4/2-1.png /content/dup4/9.png
False False
/content/dup4/2-1.png /content/dup4/1.png
False False
/content/dup4/2-1.png /content/dup4/3-3.png
False False
/content/dup4/2-1.png /content/dup4/8.png
False False
/content/dup4/2-1.png /content/dup4/2.png
True True
/content/dup4/2-1.png /content/dup4/3.png
False False
/content/dup4/2-1.png /content/dup4/1-2.png
False False
/content/dup4/2-1.png /content/dup4/12-1.png
False False
/content/dup4/2-1.png /content/dup4/11.png
False False
/content/dup4/3-3.png /content/dup4/7.png
False False
/content/dup4/3-3.png /content/dup4/5-1.png
False False
/content/dup4/3-3.png /content/dup4/3-4.png
True True
/content/dup4/3-3.png /content/dup4/6-1.png
False False
/content/dup4/3-3.png /content/dup4/11-1.png
False False
/content/dup4/3-3.png /content/dup4/12.png
False False
/content/dup4/3-3.png /content/dup4/8-1.png
False False
/content/dup4/3-3.png /content/dup4/7-1.png
False False
/content/dup4/3-3.png /content/dup4/1-1.png
False False
/content/dup4/3-3.png /content/dup4/3-1.png
True True
/content/dup4/3-3.png /content/dup4/5.png
False False
/content/dup4/3-3.png /content/dup4/10-1.png
False False
/content/dup4/3-3.png /content/dup4/6.png
False False
/content/dup4/3-3.png /content/dup4/9-1.png
False False
/content/dup4/3-3.png /content/dup4/10.png
False False
/content/dup4/3-3.png /content/dup4/4-1.png
False False
/content/dup4/3-3.png /content/dup4/3-2.png
True True
/content/dup4/3-3.png /content/dup4/4.png
image failure
False False
/content/dup4/3-3.png /content/dup4/9.png
False False
/content/dup4/3-3.png /content/dup4/1.png
False False
/content/dup4/3-3.png /content/dup4/2-1.png
False False
/content/dup4/3-3.png /content/dup4/8.png
False False
/content/dup4/3-3.png /content/dup4/2.png
False False
/content/dup4/3-3.png /content/dup4/3.png
True True
/content/dup4/3-3.png /content/dup4/1-2.png
False False
/content/dup4/3-3.png /content/dup4/12-1.png
False False
/content/dup4/3-3.png /content/dup4/11.png
False False
/content/dup4/8.png /content/dup4/7.png
False False
/content/dup4/8.png /content/dup4/5-1.png
False False
/content/dup4/8.png /content/dup4/3-4.png
False False
/content/dup4/8.png /content/dup4/6-1.png
False False
/content/dup4/8.png /content/dup4/11-1.png
False False
/content/dup4/8.png /content/dup4/12.png
False False
/content/dup4/8.png /content/dup4/8-1.png
not match
/content/dup4/8.png /content/dup4/7-1.png
False False
/content/dup4/8.png /content/dup4/1-1.png
False False
/content/dup4/8.png /content/dup4/3-1.png
False False
/content/dup4/8.png /content/dup4/5.png
False False
/content/dup4/8.png /content/dup4/10-1.png
False False
/content/dup4/8.png /content/dup4/6.png
False False
/content/dup4/8.png /content/dup4/9-1.png
False False
/content/dup4/8.png /content/dup4/10.png
False False
/content/dup4/8.png /content/dup4/4-1.png
False False
/content/dup4/8.png /content/dup4/3-2.png
False False
/content/dup4/8.png /content/dup4/4.png
image failure
False False
/content/dup4/8.png /content/dup4/9.png
False False
/content/dup4/8.png /content/dup4/1.png
False False
/content/dup4/8.png /content/dup4/2-1.png
False False
/content/dup4/8.png /content/dup4/3-3.png
False False
/content/dup4/8.png /content/dup4/2.png
False False
/content/dup4/8.png /content/dup4/3.png
False False
/content/dup4/8.png /content/dup4/1-2.png
False False
/content/dup4/8.png /content/dup4/12-1.png
False False
/content/dup4/8.png /content/dup4/11.png
False False
/content/dup4/2.png /content/dup4/7.png
False False
/content/dup4/2.png /content/dup4/5-1.png
False False
/content/dup4/2.png /content/dup4/3-4.png
False False
/content/dup4/2.png /content/dup4/6-1.png
False False
/content/dup4/2.png /content/dup4/11-1.png
False False
/content/dup4/2.png /content/dup4/12.png
False False
/content/dup4/2.png /content/dup4/8-1.png
False False
/content/dup4/2.png /content/dup4/7-1.png
False False
/content/dup4/2.png /content/dup4/1-1.png
False False
/content/dup4/2.png /content/dup4/3-1.png
False False
/content/dup4/2.png /content/dup4/5.png
False False
/content/dup4/2.png /content/dup4/10-1.png
False False
/content/dup4/2.png /content/dup4/6.png
False False
/content/dup4/2.png /content/dup4/9-1.png
False False
/content/dup4/2.png /content/dup4/10.png
False False
/content/dup4/2.png /content/dup4/4-1.png
False False
/content/dup4/2.png /content/dup4/3-2.png
False False
/content/dup4/2.png /content/dup4/4.png
image failure
False False
/content/dup4/2.png /content/dup4/9.png
False False
/content/dup4/2.png /content/dup4/1.png
False False
/content/dup4/2.png /content/dup4/2-1.png
True True
/content/dup4/2.png /content/dup4/3-3.png
False False
/content/dup4/2.png /content/dup4/8.png
False False
/content/dup4/2.png /content/dup4/3.png
False False
/content/dup4/2.png /content/dup4/1-2.png
False False
/content/dup4/2.png /content/dup4/12-1.png
False False
/content/dup4/2.png /content/dup4/11.png
False False
/content/dup4/3.png /content/dup4/7.png
False False
/content/dup4/3.png /content/dup4/5-1.png
False False
/content/dup4/3.png /content/dup4/3-4.png
True True
/content/dup4/3.png /content/dup4/6-1.png
False False
/content/dup4/3.png /content/dup4/11-1.png
False False
/content/dup4/3.png /content/dup4/12.png
False False
/content/dup4/3.png /content/dup4/8-1.png
False False
/content/dup4/3.png /content/dup4/7-1.png
False False
/content/dup4/3.png /content/dup4/1-1.png
False False
/content/dup4/3.png /content/dup4/3-1.png
True True
/content/dup4/3.png /content/dup4/5.png
False False
/content/dup4/3.png /content/dup4/10-1.png
False False
/content/dup4/3.png /content/dup4/6.png
False False
/content/dup4/3.png /content/dup4/9-1.png
False False
/content/dup4/3.png /content/dup4/10.png
False False
/content/dup4/3.png /content/dup4/4-1.png
False False
/content/dup4/3.png /content/dup4/3-2.png
True True
/content/dup4/3.png /content/dup4/4.png
image failure
False False
/content/dup4/3.png /content/dup4/9.png
False False
/content/dup4/3.png /content/dup4/1.png
False False
/content/dup4/3.png /content/dup4/2-1.png
False False
/content/dup4/3.png /content/dup4/3-3.png
not match
/content/dup4/3.png /content/dup4/8.png
False False
/content/dup4/3.png /content/dup4/2.png
False False
/content/dup4/3.png /content/dup4/1-2.png
False False
/content/dup4/3.png /content/dup4/12-1.png
False False
/content/dup4/3.png /content/dup4/11.png
False False
/content/dup4/1-2.png /content/dup4/7.png
False False
/content/dup4/1-2.png /content/dup4/5-1.png
False False
/content/dup4/1-2.png /content/dup4/3-4.png
False False
/content/dup4/1-2.png /content/dup4/6-1.png
False False
/content/dup4/1-2.png /content/dup4/11-1.png
False False
/content/dup4/1-2.png /content/dup4/12.png
False False
/content/dup4/1-2.png /content/dup4/8-1.png
False False
/content/dup4/1-2.png /content/dup4/7-1.png
False False
/content/dup4/1-2.png /content/dup4/1-1.png
True True
/content/dup4/1-2.png /content/dup4/3-1.png
False False
/content/dup4/1-2.png /content/dup4/5.png
False False
/content/dup4/1-2.png /content/dup4/10-1.png
False False
/content/dup4/1-2.png /content/dup4/6.png
False False
/content/dup4/1-2.png /content/dup4/9-1.png
False False
/content/dup4/1-2.png /content/dup4/10.png
False False
/content/dup4/1-2.png /content/dup4/4-1.png
False False
/content/dup4/1-2.png /content/dup4/3-2.png
False False
/content/dup4/1-2.png /content/dup4/4.png
image failure
False False
/content/dup4/1-2.png /content/dup4/9.png
False False
/content/dup4/1-2.png /content/dup4/1.png
True True
/content/dup4/1-2.png /content/dup4/2-1.png
False False
/content/dup4/1-2.png /content/dup4/3-3.png
False False
/content/dup4/1-2.png /content/dup4/8.png
False False
/content/dup4/1-2.png /content/dup4/2.png
False False
/content/dup4/1-2.png /content/dup4/3.png
False False
/content/dup4/1-2.png /content/dup4/12-1.png
False False
/content/dup4/1-2.png /content/dup4/11.png
False False
/content/dup4/12-1.png /content/dup4/7.png
False False
/content/dup4/12-1.png /content/dup4/5-1.png
False False
/content/dup4/12-1.png /content/dup4/3-4.png
False False
/content/dup4/12-1.png /content/dup4/6-1.png
False False
/content/dup4/12-1.png /content/dup4/11-1.png
False False
/content/dup4/12-1.png /content/dup4/12.png
True True
/content/dup4/12-1.png /content/dup4/8-1.png
False False
/content/dup4/12-1.png /content/dup4/7-1.png
False False
/content/dup4/12-1.png /content/dup4/1-1.png
False False
/content/dup4/12-1.png /content/dup4/3-1.png
False False
/content/dup4/12-1.png /content/dup4/5.png
False False
/content/dup4/12-1.png /content/dup4/10-1.png
False False
/content/dup4/12-1.png /content/dup4/6.png
False False
/content/dup4/12-1.png /content/dup4/9-1.png
False False
/content/dup4/12-1.png /content/dup4/10.png
False False
/content/dup4/12-1.png /content/dup4/4-1.png
False False
/content/dup4/12-1.png /content/dup4/3-2.png
False False
/content/dup4/12-1.png /content/dup4/4.png
image failure
False False
/content/dup4/12-1.png /content/dup4/9.png
False False
/content/dup4/12-1.png /content/dup4/1.png
False False
/content/dup4/12-1.png /content/dup4/2-1.png
False False
/content/dup4/12-1.png /content/dup4/3-3.png
False False
/content/dup4/12-1.png /content/dup4/8.png
False False
/content/dup4/12-1.png /content/dup4/2.png
False False
/content/dup4/12-1.png /content/dup4/3.png
False False
/content/dup4/12-1.png /content/dup4/1-2.png
False False
/content/dup4/12-1.png /content/dup4/11.png
False False
/content/dup4/11.png /content/dup4/7.png
False False
/content/dup4/11.png /content/dup4/5-1.png
False False
/content/dup4/11.png /content/dup4/3-4.png
False False
/content/dup4/11.png /content/dup4/6-1.png
False False
/content/dup4/11.png /content/dup4/11-1.png
True True
/content/dup4/11.png /content/dup4/12.png
False False
/content/dup4/11.png /content/dup4/8-1.png
False False
/content/dup4/11.png /content/dup4/7-1.png
False False
/content/dup4/11.png /content/dup4/1-1.png
False False
/content/dup4/11.png /content/dup4/3-1.png
False False
/content/dup4/11.png /content/dup4/5.png
False False
/content/dup4/11.png /content/dup4/10-1.png
False False
/content/dup4/11.png /content/dup4/6.png
False False
/content/dup4/11.png /content/dup4/9-1.png
False False
/content/dup4/11.png /content/dup4/10.png
False False
/content/dup4/11.png /content/dup4/4-1.png
False False
/content/dup4/11.png /content/dup4/3-2.png
False False
/content/dup4/11.png /content/dup4/4.png
image failure
False False
/content/dup4/11.png /content/dup4/9.png
False False
/content/dup4/11.png /content/dup4/1.png
False False
/content/dup4/11.png /content/dup4/2-1.png
False False
/content/dup4/11.png /content/dup4/3-3.png
False False
/content/dup4/11.png /content/dup4/8.png
False False
/content/dup4/11.png /content/dup4/2.png
False False
/content/dup4/11.png /content/dup4/3.png
False False
/content/dup4/11.png /content/dup4/1-2.png
False False
/content/dup4/11.png /content/dup4/12-1.png
False False